In [167]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, code, date, strategy, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": code,
        "date": date,
        "strategy": strategy
    }, data, upsert=True)
    

In [171]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        found.append({
            "code": code,
            "name": item["name"],
            "changes": changes
        })
        data = {
            "code": code,
            "name": item["name"],
            "date": today,
            "strategy": 1,
            "changes": changes
        }
        SaveResult(client, code, today, 1, data)

    if len(found) > 0:
        print pandas.DataFrame(found, columns=["code", "name", "changes"])
    else:
        print("No Result")
        
    client.close()

In [173]:
def Strategy_3():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10),
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    found = found[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 3
        SaveResult(client, item["code"], today, 3, data)
        
    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")
        
    client.close()

In [179]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, item["code"], today, 4, data)
        
    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount", "value"]))

    client.close()

In [223]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    print(pandas.DataFrame(found, columns=["concept", "count", "codes"]))
    
    stockDict = {}
    for k in found:
        for s in k["stocks"]:
            code = s["code"]
            if code not in stockDict:
                stockDict[code] = {
                    "code": code,
                    "name": s["name"],
                    "change": s["change"],
                    "concepts": [],
                    "date": today,
                    "strategy": 2,
                }
            stockDict[code]["concepts"].append(k["concept"])
    for code, v in stockDict.iteritems():
        SaveResult(client, code, today, 2, v)
        
    client.close()

In [227]:
def FilterSameStrategy():
    client = ConnectDB()
    today = Today()
    
    stocks = {}
    for s in client.Result.Recommend.find({"date": today}):
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": []
            }
        stocks[code]["strategies"].append(s["strategy"])
        
    result = []
    for k, v in stocks.iteritems():
        if len(v["strategies"]) > 1:
            result.append({
                "code": k,
                "name": v["name"],
                "strategies": v["strategies"]
            })
    print pandas.DataFrame(result)

In [235]:
print("Strategy 1")
Strategy_1()
print("\nStrategy 2")
Strategy_2()
print("\nStrategy 3")
Strategy_3()
print("\nStrategy 4")
Strategy_4()
print("\nSummary")
FilterSameStrategy()

Strategy 1
     code   name                                        changes
0  600601   方正科技  [10.0294985251, 10.0649350649, 1.31578947368]
1  000677   恒天海龙          [10.0, 10.0917431193, -0.30487804878]
2  002134  *ST普林  [10.0215517241, 9.95260663507, 10.0391134289]
3  300139   晓程科技   [9.96119016818, 9.9573257468, 3.07917888563]
4  300520   科大国创    [10.0039385585, 10.0086655113, 3.683737646]
5  300597   吉大通信  [10.0048804295, 9.98389694042, 9.97638724911]
6  300615   欣天科技  [10.0095328885, 9.99650471863, 9.99615532488]
7  300640   德艺文创  [9.98515219005, 10.0040832993, 10.0179694519]
8  300659   中孚信息              [10.0102774923, 9.99321727334, 0]
9  300715   凯伦股份  [10.0027862914, 9.99080600674, 10.0134861767]

Strategy 2
  concept  count                             codes
0    军工航天      4  [002338, 002214, 000561, 002023]

Strategy 3
     code    factor  name  price_5  price_today  volume_10  volume_2
0  600601  3.574554  方正科技        3         3.73     281481   1006169
1  600730  3.428404  

In [262]:
today = Today()
minDate = today + datetime.timedelta(days=-10)
collection = client.Result.Recommend
cursor = collection.find({"date": {"$gt": minDate}}) \
    .sort("date", pymongo.DESCENDING)
    
stocks = {}
daysDiff = -1
daysCount = 0
daysNeed = 3
startDate = None
for s in cursor:
    diff = (today - s["date"]).days
    if diff > daysDiff:
        daysDiff = diff
        daysCount = daysCount + 1
    if daysCount > daysNeed:
        break
    if daysCount < daysNeed:
        continue
    
    startDate = s["date"]
    code = s["code"]
    if code not in stocks:
        stocks[code] = {
            "code": code,
            "name": s["name"],
            "strategies": [s["strategy"]]
        }
    else:
        stocks[code]["strategies"].append(s["strategy"])
        
found = []
for code, stock in stocks.iteritems():
    res = []
    for s in client.Trade[code].find({"date": {"$gt": startDate}}):
        res.append(s)
    if len(res) < daysNeed - 1:
        continue
    buyPrice = res[0]["open"]
    sellPrice = res[1]["high"]
    if buyPrice == 0:
        print code, stock, res
        continue
    change = (sellPrice / buyPrice - 1) * 100
    found.append({
        "code": code,
        "name": stock["name"],
        "strategies": stock["strategies"],
        "change": change,
        "buy": buyPrice,
        "sell": sellPrice
    })
    
found = sorted(found, key=lambda item: item["change"], reverse=True)
print pandas.DataFrame(found, columns=["code", "name", "change", "buy", "sell", "strategies"])

600536 {'strategies': [2], 'code': u'600536', 'name': u'\u4e2d\u56fd\u8f6f\u4ef6'} [{u'high': 0.0, u'tradeValue': 1304162.056134, u'volume': 0, u'amount': 0, u'last_close': 26.37, u'low': 0.0, u'date': datetime.datetime(2018, 4, 26, 0, 0), u'close': 26.37, u'_id': ObjectId('5ae1921bd2df1f65abf34bc5'), u'open': 0.0}, {u'high': 26.37, u'tradeValue': 1304162.056134, u'volume': 708845, u'amount': 1797833354, u'last_close': 23.97, u'low': 23.66, u'date': datetime.datetime(2018, 4, 25, 0, 0), u'close': 26.37, u'_id': ObjectId('5ae02d7ed2df1f65abf26adb'), u'open': 23.95}]
002806 {'strategies': [1], 'code': u'002806', 'name': u'\u534e\u950b\u80a1\u4efd'} [{u'high': 0.0, u'tradeValue': 174157.199125, u'volume': 0, u'amount': 0, u'last_close': 31.01, u'low': 0.0, u'date': datetime.datetime(2018, 4, 26, 0, 0), u'close': 0.0, u'_id': ObjectId('5ae1921ed2df1f65abf3557d'), u'open': 0.0}, {u'high': 0.0, u'tradeValue': 174157.199125, u'volume': 0, u'amount': 0, u'last_close': 31.01, u'low': 0.0, u'dat